In [ ]:
# Install necessary packages
!pip install transformers datasets nltk rouge_score bert_score moverscore pyemd pytorch_pretrained_bert evaluate

In [ ]:
!pip install --upgrade numpy==1.26.0

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from transformers import PegasusTokenizer, PegasusForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from rouge_score import rouge_scorer
import evaluate

In [ ]:
# Load the original train and test dataset
train_data = pd.read_csv('/content/drive/MyDrive/mimic_train.csv')
test_data = pd.read_csv('/content/drive/MyDrive/mimic_test.csv')

In [ ]:
# Select first 3000 rows from the train dataset and 500 rows from the test dataset
train_data = train_data.iloc[:3000].copy()
test_data = test_data.iloc[:500].copy()

In [ ]:
# Preprocessing function
def preprocess(df):
    # Drop unnecessary columns
    df = df.drop(['subject_id', 'study_id'], axis=1)
    return df

In [ ]:
# Apply preprocessing to train and test datasets
train_data = preprocess(train_data)
test_data = preprocess(test_data)

In [ ]:
# Split train dataset into train and validation
train_df, val_df = train_test_split(train_data, test_size=0.2, random_state=42)

In [ ]:
# Convert to Hugging Face Dataset format
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)
test_dataset = Dataset.from_pandas(test_data)

In [ ]:
# Tokenizer and model initialization
model_name = "google/pegasus-x-base"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/2.02k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.77k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.60M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.49k [00:00<?, ?B/s]

You are using a model of type pegasus_x to instantiate a model of type pegasus. This is not supported for all configurations of models and can yield errors.


pytorch_model.bin:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-x-base and are newly initialized: ['model.decoder.embed_positions.weight', 'model.decoder.layers.0.encoder_attn.k_proj.bias', 'model.decoder.layers.0.encoder_attn.out_proj.bias', 'model.decoder.layers.0.encoder_attn.q_proj.bias', 'model.decoder.layers.0.encoder_attn.v_proj.bias', 'model.decoder.layers.0.self_attn.k_proj.bias', 'model.decoder.layers.0.self_attn.out_proj.bias', 'model.decoder.layers.0.self_attn.q_proj.bias', 'model.decoder.layers.0.self_attn.v_proj.bias', 'model.decoder.layers.1.encoder_attn.k_proj.bias', 'model.decoder.layers.1.encoder_attn.out_proj.bias', 'model.decoder.layers.1.encoder_attn.q_proj.bias', 'model.decoder.layers.1.encoder_attn.v_proj.bias', 'model.decoder.layers.1.self_attn.k_proj.bias', 'model.decoder.layers.1.self_attn.out_proj.bias', 'model.decoder.layers.1.self_attn.q_proj.bias', 'model.decoder.layers.1.self_attn.v_proj.bias', 'model.decod

generation_config.json:   0%|          | 0.00/262 [00:00<?, ?B/s]

In [ ]:
# Tokenization function
def tokenize(batch):
    inputs = tokenizer(batch['finding'], max_length=512, truncation=True, padding='max_length', return_tensors="pt")
    targets = tokenizer(batch['impression'], max_length=150, truncation=True, padding='max_length', return_tensors="pt")
    inputs['labels'] = targets['input_ids']
    return inputs

In [ ]:
# Tokenize datasets
train_dataset = train_dataset.map(tokenize, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(tokenize, batched=True, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(tokenize, batched=True, remove_columns=test_dataset.column_names)

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

Map:   0%|          | 0/600 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [ ]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    save_total_limit=1,
    logging_dir='./logs',
    logging_steps=10,
    save_steps=500
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1611: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
# Before training:
import torch
torch.cuda.empty_cache()

In [ ]:
# Trainer initialization
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer=tokenizer
)

<ipython-input-13-bd9a267dc7a4>:2: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
# Train the model
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: aninbhtry-01 (aninbhtry-01-jadavpur-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss
1,0.457200,0.341977
2,0.300500,0.291110
3,0.287700,0.276625
4,0.244200,0.271230
5,0.267900,0.268725


/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3353: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 16384, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=1500, training_loss=0.8811842642625173, metrics={'train_runtime': 2195.9184, 'train_samples_per_second': 5.465, 'train_steps_per_second': 0.683, 'total_flos': 7316724252672000.0, 'train_loss': 0.8811842642625173, 'epoch': 5.0})

In [ ]:
# Summarization of the findings from the test dataset
test_data_with_summary = test_data.copy()
test_data_with_summary['summary'] = test_data_with_summary['finding'].apply(
    lambda x: tokenizer.decode(
        model.generate(
            tokenizer(x, return_tensors="pt", max_length=512, truncation=True)['input_ids'].to(model.device),
            max_length=150, num_beams=4, length_penalty=1.5
        )[0], skip_special_tokens=True
    )
)

In [ ]:
# Convert to CSV
file_path = '/content/drive/MyDrive/ME THESIS/Predicted Summaries/Pegasus.csv'
test_data_with_summary.to_csv(file_path, index=False)

In [ ]:
# Load the CSV file containing the model generated summaries
test_data_with_summary = pd.read_csv('/content/drive/MyDrive/ME THESIS/Predicted Summaries/Pegasus.csv')

In [ ]:
# Extract ground truth (impression) and generated summaries
references = test_data_with_summary["impression"].astype(str).tolist()
predictions = test_data_with_summary["summary"].astype(str).tolist()

In [ ]:
# Initialize ROUGE scorer
rouge = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

In [ ]:
# Compute ROUGE, METEOR, and store them
rouge1_scores, rouge2_scores, rougeL_scores = [], [], []

In [ ]:
from nltk.translate.meteor_score import meteor_score

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
for ref, pred in zip(references, predictions):
    # Compute ROUGE scores
    scores = rouge.score(ref, pred)
    rouge1_scores.append(scores['rouge1'].fmeasure)
    rouge2_scores.append(scores['rouge2'].fmeasure)
    rougeL_scores.append(scores['rougeL'].fmeasure)

In [ ]:
from bert_score import score

In [ ]:
# Compute BERTScore
P, R, F1 = score(predictions, references, lang="en", verbose=False)

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Calculate averages
average_scores = {
    "ROUGE-1": sum(rouge1_scores) / len(rouge1_scores),
    "ROUGE-2": sum(rouge2_scores) / len(rouge2_scores),
    "ROUGE-L": sum(rougeL_scores) / len(rougeL_scores),
    "BERTScore-F1": F1.mean().item()
}

In [ ]:
# Print the results
print("Average Scores:")
for metric, score in average_scores.items():
    print(f"{metric}: {score:.4f}")

Average Scores:
ROUGE-1: 0.3017
ROUGE-2: 0.1687
ROUGE-L: 0.2644
BERTScore-F1: 0.8734


In [ ]:
# Load metrics
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
# Calculate METEOR
meteor_score = meteor.compute(predictions=predictions, references=references)

In [ ]:
# Print METEOR score
print("METEOR:", meteor_score)

METEOR: {'meteor': 0.33286586437343213}
